In [2]:
import pandas as pd

In [125]:
def prepare(data):
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    
    for i in range(8,29,5):
        print(i)
        started_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp], axis=1)
    
    return data

In [126]:
data = pd.read_csv("data/all_top_10/run01.csv")

In [127]:
data = prepare(data)

8
13
18
23
28


In [128]:
data.head()

,imports,buldingtime,installtime,created,started,finished,import.start,import.stop,run1.created,run1.started,...,run5.import.start,run5.import.stop,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,starttime5
0,argparse;copy;datetime;funcx;itertools;json;ma...,33.8,29.6,2024-02-24 01:46:01.942904105+00:00,2024-02-24 01:46:04.647990666+00:00,2024-02-24 01:46:09.073126174+00:00,2024-02-24 01:46:04.920405878+00:00,2024-02-24 01:46:07.935107627+00:00,2024-02-24T01:46:10.078100248Z,2024-02-24T01:46:11.228743829Z,...,2024-02-24T01:46:25.002164307Z,2024-02-24T01:46:25.446521213Z,2.705087,4.425136,3.014702,1.150644,0.951442,1.021779,1.218273,0.791984
1,argparse;copy;funcx;itertools;json;matplotlib;...,29.9,26.5,2024-02-24 02:09:36.505206886+00:00,2024-02-24 02:09:40.258796428+00:00,2024-02-24 02:09:46.050924065+00:00,2024-02-24 02:09:40.521566627+00:00,2024-02-24 02:09:44.940560231+00:00,2024-02-24T02:09:46.803715316Z,2024-02-24T02:09:47.652338465Z,...,2024-02-24T02:10:00.297477065Z,2024-02-24T02:10:00.769389974Z,3.753590,5.792128,4.418994,0.848623,0.924125,0.801321,0.927544,1.076710
2,argparse;sys,10.2,7.7,2024-02-24 00:05:31.638063327+00:00,2024-02-24 00:05:34.541170351+00:00,2024-02-24 00:05:36.181886165+00:00,2024-02-24 00:05:35.038664640+00:00,2024-02-24 00:05:35.158730894+00:00,2024-02-24T00:05:36.969313195Z,2024-02-24T00:05:38.004811892Z,...,2024-02-24T00:05:49.581944292Z,2024-02-24T00:05:49.589224895Z,2.903107,1.640716,0.120066,1.035499,1.041559,1.145632,0.959571,1.284366
3,CLEAN,9.9,6.7,2024-02-24 00:22:43.932787955+00:00,2024-02-24 00:22:46.799827019+00:00,2024-02-24 00:22:48.437580594+00:00,2024-02-24 00:22:47.405126622+00:00,2024-02-24 00:22:47.410974338+00:00,2024-02-24T00:22:49.12014949Z,2024-02-24T00:22:49.963651528Z,...,2024-02-24T00:23:02.144244308Z,2024-02-24T00:23:02.144266820Z,2.867039,1.637754,0.005848,0.843502,1.564489,1.442526,1.143089,0.756972
4,CLEAN;os,8.7,5.6,2024-02-24 00:07:08.397271981+00:00,2024-02-24 00:07:10.981056935+00:00,2024-02-24 00:07:12.645445316+00:00,2024-02-24 00:07:11.617646310+00:00,2024-02-24 00:07:11.623721815+00:00,2024-02-24T00:07:13.405962709Z,2024-02-24T00:07:14.262820498Z,...,2024-02-24T00:07:27.046705554Z,2024-02-24T00:07:27.046730058Z,2.583785,1.664388,0.006076,0.856858,1.144579,1.034260,0.925805,0.945551
